# <span style='color:SteelBlue'>P6 - Classifiez automatiquement des biens de consommation (part 4)</span>

<img src="https://logo-marque.com/wp-content/uploads/2020/11/Flipkart-Logo.png" width="300" height="200">

# <span style='background:red; color:black'>a MAJ</span>

# <span style='background:white; color:black'>Sommaire</span>

Importations des librairies utilisees dans ce notebook <br>

Paramètres d'affichage du notebook <br>

Fonctions utilisees dans ce notebook <br>

Importation des données <br>

**Etape 1 : Description des donnees**
- _1/ etc...


Conclusion

# <span style='background:blue'>Introduction</span>

L'entreprise **"Flipkart"** est une entreprise indienne qui souhaite lancer une
marketplace e-commerce. Sur cette place de marché anglophone, des vendeurs
proposent des articles à des acheteurs en postant une photo et une description.
Pour rendre l’expérience utilisateur des vendeurs (faciliter la mise en ligne
de nouveaux articles) et des acheteurs (faciliter la recherche de produits) 
la plus fluide possible, et dans l'optique d'un passage à l'échelle, 
il devient nécessaire **d'automatiser l'attribution de la catégorie** d'un 
article (actuellement effectuée manuellement par les vendeurs, donc peu 
fiable).
___
**Missions**<br>
1/ Faire une **étude de la faisabilité d’un moteur de classification**
d’article basé sur une **description** pour automatiser l’attribution
de catégorie de l’article. 

2/ Faire une **étude de la faisabilité d’un moteur de classification**
d’article basé sur une **image** pour automatiser l’attribution
de catégorie de l’article. 

3/ **Réaliser une classification supervisée** à partir des images par la mise
en place d’une data augmentation afin d’optimiser le modèle. 

4/ **Extraire les 10 premiers produits à base de « champagne »** issus de
l’API fournie dans un fichier “.csv”, contenant pour chaque produit les
données suivantes : foodId, label, category, foodContentsLabel, image.
___
Ce notebook presente la 4ème partie concernant l'extraction de produit danns
l'API...

# <span style='background:red'>a terminer</span>

___
**Source des datasets**<br>
Source : Non précisé<br>
___

<div class="alert alert-block alert-warning">
<b>Formattage du notebook au PEP8 => lancer cette ligne de code dans le
terminal VSC : </b><br>
nbqa black Berthe_Pierrick_1_notebook_exploration_112023.ipynb</div>

# <span style='background:grey'>Importations des librairies utilisees dans ce notebook</span>

In [1]:
# Librairies generales
import os.path
import sys
from IPython.display import display
import time
from datetime import datetime

# Librairies data science
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Affichage plot dans le notebook
%matplotlib inline

# Versions
print("Version des librairies utilisees :")
print("Python        : " + sys.version)
print("Matplotlib    : " + mpl.__version__)
print("NumPy         : " + np.version.full_version)
print("Pandas        : " + pd.__version__)
print("Seaborn       : " + sns.__version__)

# Nombre de coeurs
nbr_coeur = os.cpu_count()
print(f"\nNombre de coeur disponible(s) sur ce PC: {nbr_coeur} coeur(s)")

# Afficher heure lancement
maintenant = datetime.now().isoformat()
print("\nCode lance le : " + maintenant)

# Enregistrer l'heure de debut
start_time = time.time()

# Activer la verification PEP8
print("")
%load_ext pycodestyle_magic
%pycodestyle_on

Version des librairies utilisees :
Python        : 3.11.7 (tags/v3.11.7:fa7a6f2, Dec  4 2023, 19:24:49) [MSC v.1937 64 bit (AMD64)]
Matplotlib    : 3.8.2
NumPy         : 1.26.2
Pandas        : 2.1.4
Seaborn       : 0.13.0

Nombre de coeur disponible(s) sur ce PC: 12 coeur(s)

Code lance le : 2024-01-31T12:57:11.861041



# <span style='background:grey'>Paramètres d'affichage</span>

# <span style='background:grey'>Fonctions</span>

In [ ]:
def calculer_duree_notebook(start_time):
    """
    Cette procédure calcule et affiche la durée totale depuis un temps de
    début donné.

    Args:
        start_time (float): Le temps de début en secondes depuis l'époque.

    Returns:
        None
    """

    # Calculer la durée totale
    total_time = time.time() - start_time

    # Convertir en minutes et secondes
    minutes, seconds = divmod(total_time, 60)

    # Afficher la durée totale
    message = (
        "Durée totale pour faire tourner le notebook : "
        "{0} minute(s) {1} seconde(s)"
    )
    print(message.format(int(minutes), int(seconds)))

# <span style='background:grey'>Importation des donnees</span>

Importation des fichiers CSV avec Pandas

la cellule suivante permet de charger les fichiers CSV dans des dataframes 
(MAIS ELLE NE FONCTIONNE PAS)

In [4]:
import requests

url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"

querystring = {"health[0]":"alcohol-free","category[0]":"generic-foods","nutrition-type":"cooking"}

headers = {
	"X-RapidAPI-Key": "d579b71ff8msh3474fe4121852b7p1890d7jsn6a1a14d80b18",
	"X-RapidAPI-Host": "5520337"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

3:80: E501 line too long (91 > 79 characters)
5:27: E231 missing whitespace after ':'
5:42: E231 missing whitespace after ','
5:56: E231 missing whitespace after ':'
5:72: E231 missing whitespace after ','
5:80: E501 line too long (99 > 79 characters)
5:89: E231 missing whitespace after ':'
8:1: W191 indentation contains tabs
9:1: W191 indentation contains tabs


{'messages': "The host you've provided is invalid. If you have difficulties, contact the RapidAPI support team, support@rapidapi.com"}


test avec YY
Voici les clefs de YY qu'il m'a fournit pour avancer avant d'avoir un compte qui marche

Il faut trouver :

foodId, label, category, foodContentsLabel, image

In [8]:
API_KEY = 'b52ee0b5d5f28834f1cbedd57ca49692'
API_ID = 'f367f20d'
url = "https://api.edamam.com/api/food-database/v2/parser"
querystring = {"app_key": API_KEY, "app_id": API_ID, "ingr": "champagne"}
results = []

response = requests.request("GET", url, params=querystring).json()
display(response)

{'text': 'champagne',
 'parsed': [{'food': {'foodId': 'food_a656mk2a5dmqb2adiamu6beihduu',
    'label': 'Champagne',
    'knownAs': 'dry white wine',
    'nutrients': {'ENERC_KCAL': 82.0,
     'PROCNT': 0.07,
     'FAT': 0.0,
     'CHOCDF': 2.6,
     'FIBTG': 0.0},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg'}}],
 'hints': [{'food': {'foodId': 'food_a656mk2a5dmqb2adiamu6beihduu',
    'label': 'Champagne',
    'knownAs': 'dry white wine',
    'nutrients': {'ENERC_KCAL': 82.0,
     'PROCNT': 0.07,
     'FAT': 0.0,
     'CHOCDF': 2.6,
     'FIBTG': 0.0},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg'},
   'measures': [{'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_serving',
     'label': 'Serving',
     'weight': 147.0},
    {'uri': 'http://www.edamam.com/ontologies/edam

Je dois extraire un df en CSV avec ces 5 colonnes et 10 lignes

# <span style='background:blue'>Etape 1 : Présentation du dataset</span>

## <span style='background:green'>1/ Aperçu tableau</span>

Compétence OCR:

**Définir la stratégie de collecte de données en recensant les API disponibles, et réaliser la collecte
des données répondant à des critères définis via une API (interface de programmation) en prenant
en compte les normes RGPD, afin de les exploiter pour l’élaboration d’un modèle.**

- CE1 Le candidat a défini sa stratégie de collecte de données et recensé les API disponibles pour le
besoin de son projet
- CE2 Le candidat a écrit et testé une requête pour obtenir les données via l’API.
- CE3 Le candidat a récupéré les seuls champs nécessaires. Dans le cadre de ce projet, il s’agit de :
    - Des champs foodId, label, category, foodContentsLabel, image
- CE4 Le candidat applique au moins un filtre sur un des champs nécessaires pour ne collecter que les
données ayant les valeurs correspondantes sur ce ou ces champs. Dans le cadre de ce projet, il s’agit
de :
    - filtrer sur l’ingrédient (“ingr”) champagne.
- CE5 Le candidat a stocké les données collectées via l’API dans un fichier utilisable (ex. : fichier CSV ou
pickle).
- CE6 Le candidat veille au respect des normes RGPD dans toutes phases de la collecte et du stockage
des données. En particulier :
    - Il présente les 5 grands principes du RGPD
    - Il ne gère que les données nécessaires pour la finalité du projet

In [ ]:
# Afficher temps d'exécution du notebook
calculer_duree_notebook(start_time)